In [ ]:
# PRE-STEP: Install Required Dependencies
%pip install langchain
%pip install langgraph
%pip install langchain-openai
%pip install chromadb
%pip install python-dotenv

In [ ]:
# PRE-STEP: Import Core Libraries and Configure Environment
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END

# Load environment variables
load_dotenv(dotenv_path='env.txt')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Initialize models
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
embeddings = OpenAIEmbeddings()

In [ ]:
# PRE-STEP: Build the Basic Agent
# Define Agent State with memory placeholders
class AgentState(TypedDict):
    """State container for agent memory and messages"""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    working_memory: dict  # Short-term context
    episodic_recall: list  # Retrieved past experiences
    semantic_facts: dict  # Retrieved knowledge

# Initialize vector store for future memory storage
vector_store = Chroma(
    collection_name="agent_memory",
    embedding_function=embeddings,
    persist_directory="./memory_store"
)

# Create base prompt and chain
base_prompt = PromptTemplate.from_template("""
You are a helpful assistant with memory capabilities.

Current conversation:
{messages}

Please respond to the latest message.
""")

output_parser = StrOutputParser()

# Define agent node
def agent_node(state: AgentState) -> dict:
    """Core agent logic - processes messages and generates responses"""
    messages = state["messages"][-5:] if state["messages"] else []
    formatted_messages = "\n".join([
        f"{msg.type}: {msg.content}" 
        for msg in messages
    ])
    
    chain = base_prompt | llm | output_parser
    response = chain.invoke({"messages": formatted_messages})
    
    return {"messages": [("assistant", response)]}

# Build and compile the graph
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent_node)
workflow.set_entry_point("agent")
workflow.add_edge("agent", END)

app = workflow.compile()

In [ ]:
# PRE-STEP: Test the Basic Agent
# Test the agent
test_input = {
    "messages": [("user", "Hello! What's the capital of France?")]
}

result = app.invoke(test_input)
print(result["messages"][-1].content)

In [ ]:
# Step 1: Import Dependencies for Semantic Memory
from datetime import datetime
from typing import List, Dict
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain.schema import Document

In [ ]:
# Step 2: Define Semantic Fact Structure
class SemanticFact(BaseModel):
    """Structure for a semantic memory fact"""
    subject: str = Field(description="The entity or topic this fact is about")
    predicate: str = Field(description="The relationship or property")
    object: str = Field(description="The value or related entity")
    confidence: float = Field(description="Confidence score 0-1")
    source: str = Field(description="Source of this fact (user or assistant)")

def extract_semantic_facts(messages: List) -> List[SemanticFact]:
    """Extract factual knowledge from conversation"""
    prompt = PromptTemplate.from_template("""
    Analyze this conversation and extract important factual statements.
    Focus on concrete facts, preferences, and relationships mentioned.
    
    Conversation: {conversation}
    
    Extract facts in JSON format:
    {{"facts": [{{"subject": "entity", "predicate": "relationship", 
                  "object": "value", "confidence": 0.0-1.0, "source": "user or assistant"}}]}}
    
    Only extract clear, unambiguous facts. Output valid JSON only.
    """)
    
    conversation_text = "\n".join(
        f"{msg[0]}: {msg[1]}" if isinstance(msg, tuple) else f"{msg.type}: {msg.content}"
        for msg in messages
    )
    
    try:
        result = (prompt | llm | JsonOutputParser()).invoke({"conversation": conversation_text})
        return [SemanticFact(**fact_dict) for fact_dict in result.get("facts", [])]
    except Exception as e:
        print(f"Fact extraction error: {e}")
        return []

def store_semantic_facts(vector_store, facts: List[SemanticFact], user_id: str = "default"):
    """Store semantic facts in vector memory"""
    documents = [
        Document(
            page_content=f"{fact.subject} {fact.predicate} {fact.object}",
            metadata={
                "type": "semantic", "user_id": user_id,
                "subject": fact.subject, "predicate": fact.predicate,
                "object": fact.object, "confidence": fact.confidence,
                "timestamp": datetime.now().isoformat()
            }
        ) for fact in facts
    ]
    
    if documents:
        vector_store.add_documents(documents)
    return len(documents)

In [ ]:
# Step 3: Build Semantic Memory Retrieval
def retrieve_semantic_facts(vector_store, query: str, user_id: str = "default", k: int = 5):
    """Retrieve relevant semantic facts for a query"""
    results = vector_store.similarity_search(
        query=query, k=k,
        filter={"$and": [{"type": {"$eq": "semantic"}}, {"user_id": {"$eq": user_id}}]}
    )
    
    return [
        {"subject": doc.metadata.get("subject"),
         "predicate": doc.metadata.get("predicate"),
         "object": doc.metadata.get("object"),
         "confidence": doc.metadata.get("confidence", 1.0)}
        for doc in results
    ]

def format_semantic_context(facts: List[Dict]) -> str:
    """Format semantic facts for inclusion in prompt"""
    if not facts:
        return "No relevant facts found."
    
    context = "Known facts:\n" + "\n".join(
        f"- {f['subject']} {f['predicate']} {f['object']}"
        for f in facts if f.get('confidence', 1.0) > 0.7
    )
    return context if context != "Known facts:\n" else "No relevant facts found."

In [ ]:
# Step 4: Create Semantic Memory-Aware Agent
def agent_with_semantic_memory(state: AgentState) -> dict:
    """Agent that uses and updates semantic memory"""
    messages = state.get("messages", [])
    user_id = state.get("user_id", "default")
    
    # Retrieve relevant semantic facts
    semantic_context = ""
    if messages:
        latest_query = messages[-1][1] if isinstance(messages[-1], tuple) else messages[-1].content
        facts = retrieve_semantic_facts(vector_store, latest_query, user_id=user_id, k=3)
        semantic_context = format_semantic_context(facts)
    
    # Generate response with semantic knowledge
    response = (PromptTemplate.from_template("""
You are a helpful assistant with semantic memory of facts and knowledge.

{semantic_context}

Current conversation:
{messages}

Respond using relevant facts from your semantic memory when applicable.
""") | llm | output_parser).invoke({
        "semantic_context": semantic_context,
        "messages": "\n".join(
            f"{m[0]}: {m[1]}" if isinstance(m, tuple) else f"{m.type}: {m.content}"
            for m in messages[-5:]
        ) if messages else ""
    })
    
    # Extract and store new facts
    semantic_facts = {}
    if messages:
        new_facts = extract_semantic_facts(messages + [("assistant", response)][-3:])
        if new_facts:
            store_semantic_facts(vector_store, new_facts, user_id)
            semantic_facts = {"extracted": len(new_facts)}
    
    return {"messages": [("assistant", response)], "semantic_facts": semantic_facts}

# Create workflow with semantic memory
semantic_workflow = StateGraph(AgentState)
semantic_workflow.add_node("semantic_agent", agent_with_semantic_memory)
semantic_workflow.set_entry_point("semantic_agent")
semantic_workflow.add_edge("semantic_agent", END)
semantic_app = semantic_workflow.compile()

In [ ]:
# Step 5: Test Semantic Memory Functionality
def print_response(result, title):
    print(f"{title}:")
    msg = result["messages"][-1]
    print(msg[1] if isinstance(msg, tuple) else msg.content)
    if result.get("semantic_facts", {}).get("extracted"):
        print(f"\nExtracted {result['semantic_facts']['extracted']} facts")
    print("\n" + "="*50 + "\n")

# Test conversations
test_cases = [
    ("First conversation response", {
        "messages": [("user", "I'm John Smith, I work as a software engineer at TechCorp. I prefer Python for backend development and I'm allergic to shellfish.")],
        "user_id": "john_smith"
    }),
    ("Second conversation response (using semantic memory)", {
        "messages": [("user", "Can you recommend a programming language for a new web API project?")],
        "user_id": "john_smith"
    }),
    ("Third conversation response (using allergy information)", {
        "messages": [("user", "What restaurants would you recommend for a business dinner?")],
        "user_id": "john_smith"
    })
]

for title, conversation in test_cases:
    print_response(semantic_app.invoke(conversation), title)